# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846869917890                   -0.70    4.8         
  2   -7.852321474356       -2.26       -1.53    1.0   34.4ms
  3   -7.852613569312       -3.53       -2.55    1.2   34.9ms
  4   -7.852645814468       -4.49       -2.87    2.8   49.3ms
  5   -7.852646472133       -6.18       -3.17    1.0   33.6ms
  6   -7.852646678053       -6.69       -4.12    1.0   37.6ms
  7   -7.852646686650       -8.07       -4.87    2.0   44.2ms
  8   -7.852646686719      -10.16       -5.31    1.0   35.7ms
  9   -7.852646686729      -11.00       -5.76    1.2   39.1ms
 10   -7.852646686730      -11.94       -6.63    1.2   38.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846891484514                   -0.70           4.5         
  2   -7.852527801647       -2.25       -1.64   0.80    2.0    308ms
  3   -7.852636537182       -3.96       -2.72   0.80    1.0   33.7ms
  4   -7.852646446346       -5.00       -3.26   0.80    2.0   41.2ms
  5   -7.852646676300       -6.64       -4.07   0.80    1.2   35.1ms
  6   -7.852646686380       -8.00       -4.70   0.80    1.5   39.4ms
  7   -7.852646686712       -9.48       -5.55   0.80    1.8   37.2ms
  8   -7.852646686730      -10.77       -6.57   0.80    2.0   39.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.388460e+01     4.010556e+00
 * time: 0.6903820037841797
     1     1.528494e+00     2.323914e+00
 * time: 0.9513559341430664
     2    -1.736442e+00     2.784935e+00
 * time: 0.9864509105682373
     3    -3.506905e+00     2.507857e+00
 * time: 1.0381369590759277
     4    -4.953867e+00     2.178403e+00
 * time: 1.086068868637085
     5    -6.521922e+00     1.315732e+00
 * time: 1.1375188827514648
     6    -7.307407e+00     4.344201e-01
 * time: 1.184251070022583
     7    -7.607939e+00     1.620868e-01
 * time: 1.2214488983154297
     8    -7.731953e+00     1.704794e-01
 * time: 1.2560088634490967
     9    -7.795440e+00     8.059278e-02
 * time: 1.2897629737854004
    10    -7.827055e+00     5.858020e-02
 * time: 1.3247220516204834
    11    -7.843600e+00     3.395274e-02
 * time: 1.3585259914398193
    12    -7.849273e+00     2.160845e-02
 * time: 1.394057035446167
    13    -7.851438e+00     1.302167e-02
 * time: 1.4292719364166

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846896437175                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645906132                   -1.64         
  2   -7.852646686730       -6.11       -3.71    2.27s
  3   -7.852646686730      -13.31       -7.25    312ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.7380869422792213e-7
|ρ_newton - ρ_scfv| = 2.5318272370947233e-7
|ρ_newton - ρ_dm|   = 7.542186196452146e-10
